In [20]:
#sudo apt-get update
#sudo apt-get install libleptonica-dev tesseract-ocr tesseract-ocr-dev libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn
#sudo apt-get install tesseract-ocr
#sudo apt install libtesseract-dev

#pip install pandas

In [1]:
from PIL import Image
import pytesseract
import pandas as pd
import numpy as np 
import csv
import os


In [2]:
# define document type IDs with labels

DocType_Dict = {0 : 'Letter',
                1 : 'Form',
                2 : 'E-mail',
                3 : 'Handwritten Note',
                4 : 'Advertisement',
                5 : 'scientific report',
                6 : 'scientific publication',
                7 : 'Functional specification',
                8 : 'File Folder',
                9 : 'News Article',
                10 : 'Budget',
                11 : 'Invoice',
                12 : 'Presentation',
                13 : 'Questionnaire',
                14 : 'Resume',
                15 : 'Memo'}

#df.to_csv("name of csv") #index=False, header=false)

In [23]:
df = pd.read_csv("train.txt", sep=" ", header=None) 
df.columns = ['path','doctype']
df['doctype_label']=df['doctype'].map(DocType_Dict)
df

,path,doctype,doctype_label
0,imagesq/q/o/c/qoc54c00/80035521.tif,15,Memo
1,imagese/e/w/c/ewc23d00/513280028.tif,1,Form
2,imagesw/w/b/t/wbt26e00/2053453161.tif,7,Functional specification
3,imagesm/m/k/m/mkm05e00/2040792992_2040792994.tif,10,Budget
4,imageso/o/e/x/oex80d00/522787731+-7732.tif,3,Handwritten Note
...,...,...,...
319995,imagesu/u/p/p/upp04f00/0000282789.tif,9,News Article
319996,imagesa/a/c/z/acz60f00/0011972032.tif,15,Memo
319997,imagesu/u/j/m/ujm20a00/10155388.tif,6,scientific publication
319998,imagesd/d/r/r/drr93f00/0000343578.tif,9,News Article


In [24]:
df_a = df.loc[df.path.str.contains('imagesa')]
df_a = df_a.reset_index()
df_a

,index,path,doctype,doctype_label
0,68,imagesa/a/p/x/apx86e00/2057673032.tif,7,Functional specification
1,101,imagesa/a/f/l/afl14c00/2069723960.tif,7,Functional specification
2,118,imagesa/a/j/i/aji14a00/94503027.tif,1,Form
3,120,imagesa/a/g/i/agi30c00/corti0007283.tif,9,News Article
4,142,imagesa/a/p/n/apn03f00/tob00301.79.tif,4,Advertisement
...,...,...,...,...
11797,319885,imagesa/a/u/v/auv31a00/0071040778.tif,8,File Folder
11798,319890,imagesa/a/k/u/aku93e00/2040698107.tif,10,Budget
11799,319959,imagesa/a/c/t/act93d00/508370002.tif,11,Invoice
11800,319989,imagesa/a/h/q/ahq85e00/2025854118_2025854119.tif,9,News Article


In [25]:
# Looping through images in folder and doing OCR
# Applying following cleanup steps:
# 1. Removing '\n' and '\t'
# 2. Removing data where text length is 25 characters or less 
# 


counter = 0
ocr_text = []
for index, row in df_a.iterrows():
    try:
        img = Image.open('/home/sunitc/work/images/' + row['path'])
        txt = pytesseract.image_to_string(img)
        if "\n" in txt:
            txt = txt.replace("\n", " ")
        if "\t" in txt:
            txt = txt.replace("\t", " ")
        if len(txt) > 25:
            ocr_text.append([row['doctype_label'],txt])
            counter = counter + 1
            print('\r' + str(counter), end='')
            # putting in counter so I an limit the number of rows in DS
            if counter > 2000:
                break
    except: # do nothing - keep on looping
        print('\r Error', end='')


2001

In [26]:

#ocr_text[:10]
ocr_df = pd.DataFrame(ocr_text)
ocr_df.columns = ['doc_type','doc_text']
ocr_df.head()


,doc_type,doc_text
0,Functional specification,CABARRUS COUNT? CLNISUED_FILTES BOR LES...
1,Functional specification,Loursvrite ‘BASIC EIGHT SPECIFICATION » CAMB...
2,Form,SPECIAL PROMOTION EVALUATION 19 Septemb...
3,News Article,ump urspuspUL sormboe [OurUTy nsnp...
4,Advertisement,yap10q alIym e Yim PUNO.BYDEq xOEIG UO AdAi O1...


In [27]:
ocr_df.to_csv("Document_Type_DS.csv") #index=False, header=false)

In [28]:
print("Num of Doc Types in DataSet")
ocr_df['doc_type'].value_counts()

Num of Doc Types in DataSet


Letter                      148
scientific publication      141
Resume                      137
scientific report           137
News Article                133
Memo                        133
Invoice                     132
Questionnaire               132
Form                        131
Presentation                129
Functional specification    124
E-mail                      124
Budget                      116
Handwritten Note            113
Advertisement               100
File Folder                  71
Name: doc_type, dtype: int64